In [1]:
from sklearn import svm 
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
%matplotlib inline

#tree
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from IPython.display import Image 
from IPython.display import IFrame 
import pydot_ng as pydot

import warnings
warnings.simplefilter("ignore")

In [2]:
#read the data
match = pd.read_csv("DataSetForClassification.csv")


In [3]:
match.columns

Index(['Unnamed: 0', 'Season', 'Date', 'Home_Team_ID', 'Away_Team_ID',
       'Home_Goals', 'Away_Goals', 'home_player_1', 'home_player_2',
       'home_player_3', 'home_player_4', 'home_player_5', 'home_player_6',
       'home_player_7', 'home_player_8', 'home_player_9', 'home_player_10',
       'home_player_11', 'away_player_1', 'away_player_2', 'away_player_3',
       'away_player_4', 'away_player_5', 'away_player_6', 'away_player_7',
       'away_player_8', 'away_player_9', 'away_player_10', 'away_player_11',
       'HomeTeamRating', 'AwayTeamRating', 'HomeShortPass', 'AwayShortPass',
       'HomeLongPass', 'AwayLongPass', 'HomeFreeKickAccuracy',
       'AwayFreeKickAccuracy', 'HomeBallControl', 'AwayBallControl',
       'HomeSprintSpeed', 'AwaySprintSpeed', 'HomeShotPower', 'AwayShotPower',
       'HomeInterception', 'AwayInterception', 'HomePenalties',
       'AwayPenalties', 'HomeStandingTackle', 'AwayStandingTackle',
       'HomeSlindingTackle', 'AwaySlindingTackle', 'Home Supp

In [4]:
#drop the columns that won't be used for prediction
match.drop("Unnamed: 0", axis = 1, inplace = True)
match.drop("Home_Goals", axis = 1, inplace = True)
match.drop("Away_Goals", axis = 1, inplace = True)
match.drop("HomeVsAway_Home", axis = 1, inplace = True)
match.drop("HomeVsAway_Away", axis = 1, inplace = True)
match.drop(match.ix[:, "home_player_1": "away_player_11"], axis = 1, inplace = True)

In [14]:
#split the data into training and validation set
#season 2008-09 to 2014-15 will be the training data
#we will predict the outcome of the matches in 2015-16
#train = match.loc[match["Season"] != "2015/2016",]
train = match.loc[~match["Season"].isin(["", "2015/2016"]),]
validation = match.loc[match["Season"] == "2015/2016",]

#trees
X = train.loc[:, "HomeTeamRating":"AwaydefenceTeamWidth"]
Y =  train['Outcome']
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X,Y)

#predict - try an out of sample prediction
X = validation.ix[:, "HomeTeamRating":"AwaydefenceTeamWidth"]
Y = validation['Outcome']

validation['prediction'] = clf.predict(X)

#prediction errors
print((validation['prediction'] != validation['Outcome']).sum())

#accuracy
print((validation['prediction'] == validation['Outcome']).sum() / len(validation['Outcome']))

233
0.386842105263


In [15]:
#random forest on train data
X = train.ix[:, "HomeTeamRating":"AwaydefenceTeamWidth"]
Y =  train['Outcome']
clf_rf = RandomForestClassifier(max_features="log2")
clf_rf = clf_rf.fit(X, Y)
clf_rf_scores = cross_val_score(clf_rf, X, Y, cv=5)

#try an out of sample prediction
X = validation.ix[:, "HomeTeamRating":"AwaydefenceTeamWidth"]
Y = validation['Outcome']
validation['prediction'] = clf_rf.predict(X)

#prediction errors
print((validation['prediction'] != validation['Outcome']).sum())

#accuracy
print((validation['prediction'] == validation['Outcome']).sum() / len(validation['Outcome']))

212
0.442105263158


In [16]:
#try boosting
X = train.ix[:, "HomeTeamRating":"AwaydefenceTeamWidth"]
Y =  train['Outcome']
clf = AdaBoostClassifier()
clf = clf.fit(X,Y)
clf.score(X,Y)

#try an out of sample prediction
X = validation.ix[:, "HomeTeamRating":"AwaydefenceTeamWidth"]
Y = validation['Outcome']
validation['prediction'] = clf.predict(X)

#prediction errors
print((validation['prediction'] != validation['Outcome']).sum())

#accuracy
print((validation['prediction'] == validation['Outcome']).sum() / len(validation['Outcome']))

223
0.413157894737
